In [ ]:
!pip install hazm
!pip install transformers

     |████████████████████████████████| 317kB 6.4MB/s 
     |████████████████████████████████| 235kB 9.0MB/s 
     |████████████████████████████████| 1.4MB 10.2MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=155187 sha256=a32a429ff8e710bd522eecea5cc1a0124948a7a64f888c3a51c3e89f436d1b38
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394467 sha256=3c33b562d296df5fcbc3d9793946bab5b6cd68a80d30914f9bfc9269e5e9a2a6
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 1.8MB 6.0MB/s 
     |████████████████████████████████| 2.9MB 19.6MB/s 
     |████████████████████████████████| 890kB 4

In [ ]:
import numpy as np
import pandas as pd

import hazm

import transformers 
from transformers import AutoTokenizer, AutoConfig
from transformers import TFAutoModelForTokenClassification
import tensorflow as tf

In [ ]:
peyma_translate = {
    "B_DAT": "تاریخ",
    "B_LOC": "موقعیت",
    "B_MON": "پول",
    "B_ORG": "سازمنان",
    "B_PCT": "درصد",
    "B_PER": "شخص",
    "B_TIM": "زمان",
    "I_DAT": "تاریخ",
    "I_LOC": "موقعیت",
    "I_MON": "پول",
    "I_ORG": "سازمان",
    "I_PCT": "درصد",
    "I_PER": "شخص",
    "I_TIM": "زمان",
    "O": None,
}
arman_translate = {
    "B-event": "رویداد",
    "B-fac": "امکانات",
    "B-loc": "موقعیت",
    "B-org": "سازمان",
    "B-pers": "شخص",
    "B-pro": "محصول",
    "I-event": "رویداد",
    "I-fac": "امکانات",
    "I-loc": "موقعیت",
    "I-org": "سازمان",
    "I-pers": "شخص",
    "I-pro": "محصول",
    "O": None
}

ner_translate = {
    "B-date": "تاریخ",
    "B-event": "رویداد",
    "B-facility": "امکانات",
    "B-location": "موقعیت",
    "B-money": "پول",
    "B-organization": "سازمان",
    "B-person": "شخص",
    "B-product": "محصول",
    "B-time": "زمان",
    "B-percent": "درصد",
    "I-date": "تاریخ",
    "I-event": "رویداد",
    "I-facility": "امکانات",
    "I-location": "موقعیت",
    "I-money": "پول",
    "I-organization": "سازمان",
    "I-person": "شخص",
    "I-product": "محصول",
    "I-time": "زمان",
    "I-percent": "درصد",
    "O": None
}

In [ ]:
texts = [
    "مدیرکل محیط زیست استان البرز با بیان اینکه با بیان اینکه موضوع شیرابه‌های زباله‌های انتقال یافته در منطقه حلقه دره خطری برای این استان است، گفت: در این مورد گزارشاتی در ۲۵ مرداد ۱۳۹۷ تقدیم مدیران استان شده است.",
    "به گزارش خبرگزاری تسنیم از کرج، حسین محمدی در نشست خبری مشترک با معاون خدمات شهری شهرداری کرج که با حضور مدیرعامل سازمان‌های پسماند، پارک‌ها و فضای سبز و نماینده منابع طبیعی در سالن کنفرانس شهرداری کرج برگزار شد، اظهار داشت: ۸۰٪  جمعیت استان البرز در کلانشهر کرج زندگی می‌کنند.",
    "وی افزود: با همکاری‌های مشترک بین اداره کل محیط زیست و شهرداری کرج برنامه‌های مشترکی برای حفاظت از محیط زیست در شهر کرج در دستور کار قرار گرفته که این اقدامات آثار مثبتی داشته و تاکنون نزدیک به ۱۰۰ میلیارد هزینه جهت خریداری اکس-ریس صورت گرفته است.",
]


In [ ]:


class NER:
  def __init__(self,model_path):
    self.normalizer = hazm.Normalizer()
    self.model, self.tokenizer, self.labels = parsbert_ner_load_model(model_path)

  def cleanize(self,text):
      """A way to normalize and even clean the text"""
      # clean text
      # do some fns
      return self.normalizer.normalize(text)


  def parsbert_ner_load_model(model_name):
      """Load the model"""
      try:
          config = AutoConfig.from_pretrained(model_name)
          tokenizer = AutoTokenizer.from_pretrained(model_name)
          model = TFAutoModelForTokenClassification.from_pretrained(model_name)
          labels = list(config.label2id.keys())
          return model, tokenizer, labels
      except:
          return [None] * 3


  def predict(self,texts):
      output_predictions = []
      for sequence in texts:
          sequence = self.cleanize(sequence)
          tokens = self.tokenizer.tokenize(self.tokenizer.decode(self.tokenizer.encode(sequence)))
          inputs = self.tokenizer.encode(sequence, return_tensors="tf")
          outputs = self.model(inputs)[0]
          predictions = tf.argmax(outputs, axis=2)
          predictions = [(token, self.labels[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())]
          output_predictions.append(predictions)

      return output_predictions
    
  def predict_filtered(self,texts):
    output_predictions = self.predict(texts)
    return [[pair for pair in array if pair[1] != 'O'] for array in output_predictions]

  def predict_concat(self,texts):
      output_filtered = self.predict_filtered(texts)
      result = []
      for array in output_filtered:
        tokens_concat = []
        current_token = ''
        current_cat = ''
        for token,cat in array:
          if cat[0] == 'B':
            if current_token != '': tokens_concat.append((current_token,current_cat))
            current_token = token
            current_cat = cat[2:]
          elif cat[0] == 'I':
            current_token = current_token + ' '+ token
        if current_token != '': tokens_concat.append((current_token,current_cat))
        result.append(tokens_concat)
      return result

In [ ]:
#initializing ner class
ner = NER('HooshvareLab/bert-base-parsbert-ner-uncased')

Some layers from the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased were not used when initializing TFBertForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [ ]:
ner.predict_concat(texts)

[[('محیط زیست استان البرز', 'organization'),
  ('منطقه حلقه دره', 'location'),
  ('۲۵ مرداد ۱۳۹۷', 'date')],
 [('خبرگزاری تسنیم', 'organization'),
  ('کرج', 'location'),
  ('حسین محمدی', 'person'),
  ('شهرداری کرج', 'organization'),
  ('سازمانهای پسماند ، پارکها و فضای سبز', 'organization'),
  ('منابع طبیعی', 'organization'),
  ('سالن کنفرانس شهرداری کرج', 'location'),
  ('۸۰ ٪', 'percent'),
  ('استان البرز', 'location'),
  ('کرج', 'location')],
 [('اداره کل محیط زیست', 'organization'),
  ('شهرداری کرج', 'organization'),
  ('شهر کرج', 'location'),
  ('۱۰۰ میلیارد', 'money')]]